In [1]:
import a_mps
import my_mps

import d_dmrg
from bosehubbardfile import BoseHubbardModel
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
B = np.zeros([1, (5+1), 1], np.float)
B[0, 0, 0] = 1.
S = np.ones([1], np.float)
Bs = [B.copy() for i in range(5)]
Ss = [S.copy() for i in range(5)]
a = np.tensordot(np.diag(Ss[0]), Bs[0], [1, 0])
print(a)
b = np.tensordot(a, Bs[1], [2, 0])
print(b)

In [ ]:
model = BoseHubbardModel(5, 0.1, 1.0, 0.4, 5)
print(model.H_bonds[1])
#print(np.tensordot(model.H_bonds[1], b, axes=[[2, 3], [1, 2]]))

In [ ]:
energyss = []

In [ ]:
def e_dmrg_gs(L, t, U, mu, nc, max_N_sweeps=20, chi_max=50, eps=1.e-10):
    model = BoseHubbardModel(L, t, U, mu, nc)
    psi = my_mps.init_spinup_MPS(L, nc)
    eng = d_dmrg.DMRGEngine(psi, model, chi_max=chi_max, eps=eps)
    Eprev = np.inf
    for sw in range(max_N_sweeps):
        for i in range(L-2):
            E = eng.sweep()
        #E = model.energy(psi)
            E = eng.update_bond(i)
            #if abs((E - Eprev)) < eps: 
             #   print('')  # newline
              #  break
            #print(E.dtype)
            #np.array(E)
            #energyss.append(E)
            #Eprev = E
    print("max final bond dimension: ", max(psi.get_chi()))
    return model, psi, E

In [ ]:
#model = BoseHubbardModel(5, 0.1, 1.0, 0.4, 4)
model, psi, E = e_dmrg_gs(10, 0.1, 1.0, 0.4, 4)

print(E)

In [2]:
def my_expectation_value(psi, model):
    for i in range(model.L):
        ket = psi[i]
        bra = np.conj(psi[i])
        contraction = np.tensordot(ket, model.fixed_H_mpo[i], (1,2)) #vL [i] vR , hL hR [hup] hdown
        nextcontraction = np.tensordot(contraction, bra, axes=([4], [1])) #vL vR hL hR [hdown] , vL [i] vR
        #firstcontraction.append(nextcontraction)
        if i == 0: 
            energy = nextcontraction
        else: 
            #print("Energy", energy.shape)
            #print("Contraction", nextcontraction.shape)
            energy = np.tensordot(energy, nextcontraction, axes=([1,3,5],[0,2,4])) #vL [vR] hL [hR] vL [vR],  [vL] vR [hL] hR [vL] vR
            energy = np.transpose(energy, (0,3,2,4,1,5))
    return np.squeeze(energy)

In [3]:
import copy
def dmrg_gs(L, t, U, mu, nc, max_N_sweeps=100, chi_max=50, eps=1.e-10):
    model = BoseHubbardModel(L, t, U, mu, nc)
    psi = my_mps.init_spinup_MPS(L, nc)
    psi_initial = copy.deepcopy(psi.Bs)
    eng = d_dmrg.DMRGEngine(psi, model, chi_max=chi_max, eps=eps)
    Eprev = np.inf
    energyss = []
    for i in range(max_N_sweeps):
        print("sweep", i)
        eng.sweep()
        E = my_expectation_value(psi.Bs, model)
        #if abs((E - Eprev)) < eps: 
        #    print('')  # newline
        #    break
        #print(E)
        energyss.append(E)
        #Eprev = E
    print("max final bond dimension: ", max(psi.get_chi()))
    return model, psi, energyss

In [10]:
# model, psi, energyss = dmrg_gs(3, 0.1, 1.0, 0.4, 4)
energyss = np.array(energyss)

Es = []
for L in range(2, 20):
    model, psi, energyss = dmrg_gs(L, 0.5, 0, 0, 3)
    Es = my_expectation_value(psi.Bs, model)
    print(energyss[-1])
    print(Es)
Es.append(Es)

#print(model.energy(psi))

Es = np.array(Es)

plt.figure()
#print(energyss)
plt.plot(Es, marker='*')
plt.xlabel('step')
plt.ylabel('energies')
plt.show()


sweep 0
sweep 1
sweep 2
sweep 3
sweep 4
sweep 5
sweep 6
sweep 7
sweep 8
sweep 9
sweep 10
sweep 11
sweep 12
sweep 13
sweep 14
sweep 15
sweep 16
sweep 17
sweep 18
sweep 19
sweep 20
sweep 21
sweep 22
sweep 23
sweep 24
sweep 25
sweep 26
sweep 27
sweep 28
sweep 29
sweep 30
sweep 31
sweep 32
sweep 33
sweep 34
sweep 35
sweep 36
sweep 37
sweep 38
sweep 39
sweep 40
sweep 41
sweep 42
sweep 43
sweep 44
sweep 45
sweep 46
sweep 47
sweep 48
sweep 49
sweep 50
sweep 51
sweep 52
sweep 53
sweep 54
sweep 55
sweep 56
sweep 57
sweep 58
sweep 59
sweep 60
sweep 61
sweep 62
sweep 63
sweep 64
sweep 65
sweep 66
sweep 67
sweep 68
sweep 69
sweep 70
sweep 71
sweep 72
sweep 73
sweep 74
sweep 75
sweep 76
sweep 77
sweep 78
sweep 79
sweep 80
sweep 81
sweep 82
sweep 83
sweep 84
sweep 85
sweep 86
sweep 87
sweep 88
sweep 89
sweep 90
sweep 91
sweep 92
sweep 93
sweep 94
sweep 95
sweep 96
sweep 97
sweep 98
sweep 99
max final bond dimension:  1
0.0
0.0
sweep 0
sweep 1
sweep 2
sweep 3
sweep 4
sweep 5
sweep 6
sweep 7
sweep 8
s

KeyboardInterrupt: 

In [35]:
def new_energy(psi, fixed_H_mpo):
    for i in range(model.L):
        ket = psi[i]
        bra = np.conj(psi[i])
        contraction = np.tensordot(ket,fixed_H_mpo[i], (1,2)) #vL [i] vR , hL hR [hup] hdown
        nextcontraction = np.tensordot(contraction, bra, axes=([4], [1])) #vL vR hL hR [hdown] , vL [i] vR
        #firstcontraction.append(nextcontraction)
        if i == 0: 
            energy = nextcontraction
        else: 
            print("Energy", energy.shape)
            print("Contraction", nextcontraction.shape)
            energy = np.tensordot(energy, nextcontraction, axes=([1,3,5],[0,2,4])) #vL [vR] hL [hR] vL [vR],  [vL] vR [hL] hR [vL] vR
            energy = np.transpose(energy, (0,3,2,4,1,5))
    return np.squeeze(energy)



In [ ]:
new_energy(psi, model.fixed_H_mpo)

In [ ]:

def contract_right_block(A, W, R):
    """
    Contraction step from right to left, with a matrix product operator
    sandwiched in between.

    To-be contracted tensor network::

          _____           _________
         /     \         /         \
      ---|1 A 2|---   ---|0        |
         \__0__/         |         |
            |            |         |
                         |         |
          __|__          |         |
         /  1  \         |         |
      ---|2 W 3|---   ---|1   R    |
         \__0__/         |         |
            |            |         |
                         |         |
          __|__          |         |
         /  0  \         |         |
      ---|1 A*2|---   ---|2        |
         \_____/         \_________/
    """

    assert A.ndim == 3
    assert W.ndim == 4
    assert R.ndim == 3

    # multiply with A tensor
    T = np.tensordot(A, R, 1)

    # multiply with W tensor
    T = np.tensordot(W, T, axes=((1, 3), (0, 2)))

    # interchange levels 0 <-> 2 in T
    T = T.transpose((2, 1, 0, 3))

    # multiply with conjugated A tensor
    Rnext = np.tensordot(T, theta.conj(), axes=((2, 3), (0, 2)))

    return Rnext

def operator_average(op:fixed_H_MPO, psi:psi):
    """
    Compute the expectation value `<psi | op | psi>`.

    Args:
        psi: wavefunction represented as MPS
        op:  operator represented as MPO

    Returns:
        complex: `<psi | op | psi>`
    """

    assert psi.L == len(op)

    if psi.L == 0:
        return 0

    # initialize T by identity matrix
    T = np.identity(psi.theta[-1].shape[2], dtype=psi.dtype)
    T = np.reshape(T, (psi.theta[-1].shape[2], 1, psi.theta[-1].shape[2]))

    for i in reversed(range(psi.L)):
        T = contract_right_block(psi.theta[i], op.theta[i], T)

    # T should now be a 1x1x1 tensor
    assert T.shape == (1, 1, 1)

    return T[0, 0, 0]
    

In [ ]:
mus = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
ts = [0.0, 0.05, 0.1, 0.15, 0.2, 0.21, 0.23, 0.25, 0.3, 0.35]
Us = [1.0] #0.3, 0.4, 0.5, 0.8, 1.0, 1.1, 1.2, 1.5, 2.0, 5.0

In [ ]:

plt.figure()
Es = []
t_U = []
mu_U = []
for U in Us:
    for mu in mus:
        print('-'*40)
        for t in ts:
            print("U =", U)
            print("mu =", mu)
            print("t =", t)
            t_U.append(t/U)
            mu_U.append(mu/U)
            model, psi = dmrg_gs(9, t, U, mu, 4, chi_max=64, eps=1.e-10)
            Es.append(my_expectation_value(psi.Bs, model))
    Es = np.array(Es)
    t_U = np.array(t_U)
    mu_U = np.array(mu_U)
    print(t_U)
    print(mu_U)
    Es = np.reshape(Es, (len(mus), len(ts)))
    print("max chi = ", max(psi.get_chi()))
plt.yticks(ticks=np.arange(len(t_U)),labels=t_U,rotation=90)
plt.xticks(ticks=np.arange(len(mu_U)),labels=mu_U)
# save this plot inside a variable called hm
hm=plt.imshow(Es, cmap='Blues',interpolation="nearest")
# pass this heatmap object into plt.colorbar method.
plt.colorbar(hm)
plt.show()

In [ ]:
print(Es)
print(t_U)
print(mu_U)

In [ ]:
Es = np.reshape(Es, (len(mus), len(ts)))
print(Es)

In [ ]:
plt.figure()
mu_U_lbl = np.unique(mu_U)
plt.xticks(ticks=np.arange(len(t_U)),labels=t_U,rotation=90)
plt.yticks(ticks=np.arange(len(mu_U)),labels=mu_U_lbl)
# save this plot inside a variable called hm
xmin = 0
xmax = 9
ymin = 0
ymax = 10
hm=plt.imshow(Es, cmap='Blues',interpolation="nearest", extent = (xmin,xmax,ymin,ymax))
# pass this heatmap object into plt.colorbar method.
plt.colorbar(hm)
plt.show()

In [ ]:
plt.figure()
for U in [1.0]:
    Es = []
    for mu in mus:
        print('-'*40)
        for t in ts:
            print("U =", U)
            print("mu =", mu)
            print("t =", t)
            model, psi = dmrg_gs(7, t, U, mu, 5, chi_max=64, eps=1.e-10)
            Es.append(model.energy(psi))
    Es = np.array(Es)
    Es = np.reshape(Es, (len(mus), len(ts)))
    print("max chi = ", max(psi.get_chi()))
plt.yticks(ticks=np.arange(len(mus)),labels=mus,rotation=90)
plt.xticks(ticks=np.arange(len(ts)),labels=ts)
# save this plot inside a variable called hm
hm=plt.imshow(Es, cmap='Blues',interpolation="nearest")
# pass this heatmap object into plt.colorbar method.
plt.colorbar(hm)
plt.show()



plt.figure()
Es = []
for U in Us:
    for mu in [1.0]:
        print('-'*40)
        for t in ts:
            print("U =", U)
            print("mu =", mu)
            print("t =", t)
            model, psi = dmrg_gs(7, t, U, mu, 5, chi_max=64, eps=1.e-10)
            Es.append(model.energy(psi))
Es = np.array(Es)
Es = np.reshape(Es, (len(Us), len(ts)))
print("max chi = ", max(psi.get_chi()))
plt.yticks(ticks=np.arange(len(mus)),labels=mus,rotation=90)
plt.xticks(ticks=np.arange(len(ts)),labels=ts)
# save this plot inside a variable called hm
hm=plt.imshow(Es, cmap='Blues',interpolation="nearest")
# pass this heatmap object into plt.colorbar method.
plt.colorbar(hm)
plt.show()



plt.figure()
for U in Us:
    Es = []
    for mu in mus:
        print('-'*40)
        for t in [1.0]:
            print("U =", U)
            print("mu =", mu)
            print("t =", t)
            model, psi = dmrg_gs(7, t, U, mu, 5, chi_max=64, eps=1.e-10)
            Es.append(model.energy(psi))
Es = np.array(Es)
Es = np.reshape(Es, (len(Us), len(mus)))
print("max chi = ", max(psi.get_chi()))
plt.yticks(ticks=np.arange(len(mus)),labels=mus,rotation=90)
plt.xticks(ticks=np.arange(len(ts)),labels=ts)
# save this plot inside a variable called hm
hm=plt.imshow(Es, cmap='Blues',interpolation="nearest")
# pass this heatmap object into plt.colorbar method.
plt.colorbar(hm)
plt.show()


In [ ]:
plt.figure()
U_ss = [0.0, 0.5, 1.0, 2.0]
Es = []
for U in U_ss:
    print("U =", U)
    model, psi = dmrg_gs(5, 1.5, U, 0.4, 5, chi_max=64, eps=1.e-10)
    Es.append(model.energy(psi))
Es = np.array(Es)
print(Es)
U_ss = np.array(U_ss)
print("max chi = ", max(psi.get_chi()))
plt.plot(U_ss, Es, marker='o')
#plt.xscale('log')
plt.xlabel('U')
plt.ylabel('energy')
plt.legend()
plt.show()

In [ ]:
#Correlation function section 


# e)
def correlation_function(psi, X, Y, i):
    L = psi.L
    assert i < L
    corrs = []
    # first case: j = i
    # Note: psi.get_theta(i) = Lambda[i] B[i] with axes vL i vR
    theta = psi.get_theta1(i)  # vL i vR
    X_Y_theta = np.tensordot(np.dot(X, Y), # i [i*], [i] i*
                             theta, axes=([1], [1]))  # i [i*], vL [i] vR
    contr = np.tensordot(theta.conj(), X_Y_theta, axes=([0, 1, 2], [1, 0, 2]))  # [vL*] [i*] [vR*], [i] [vL] [vR]
    corrs.append(contr)
    # now for i < j
    theta = psi.get_theta1(i)  # vL i vR
    LP = np.tensordot(X, theta, axes=([1], [1]))  # i [i*], vL [i] vR
    LP = np.tensordot(theta.conj(), LP, axes=([0, 1], [1, 0])) # [vL*] [i*] vR*, [i] [vL] vR
    # now LP has axes vR*, vR
    for j in range(i+1, L):
        # contract with B Y B* to evaluate the correlation function for given j
        B = psi.Bs[j]  # vL j vR
        Y_B = np.tensordot(Y, B, axes=([1], [1])) # j [j*], vL [j] vR
        contr = np.tensordot(LP, Y_B, axes=([1], [1])) # vR* [vR], j [vL] vR
        contr = np.tensordot(contr, B.conj(), axes=([0, 1, 2], [0, 1, 2])) # [vR*] [j] [vR], [vL*] [j*] [vR*]
        corrs.append(contr.item())
        # get next LP for next i
        LP = np.tensordot(LP, B, axes=([1], [0])) # vR* [vR], [vL] j vR
        LP = np.tensordot(B.conj(), LP, axes=([0, 1], [0, 1])) # [vL*] [j*] vR*, [vR*] [j] vR
    return np.real_if_close(corrs)


# just to test it
model, psi = dmrg_gs(5, 0.1, 1.0, 1.0, 4, chi_max=64, eps=1.e-10)
cr = model.init_creation_op() 
ann = model.init_annihilation_op()
correlation_fn = correlation_function(psi, cr, ann, model.L//2)

print(correlation_fn)

In [ ]:
num = model.init_number_op()
id = np.eye(model.d)
density = correlation_function(psi, num, id, model.L//2)

print(density)

In [ ]:
# calculate correlations
corrs = {}
#conn_corrs = {} dont think we need 

#what is keys and what should be varied for correlations - creation annihilation?
# vary nc ?? 
#superfluid order? 

#or plot graph of corr against H parameter eg t or t/u ot t/mu?

#Compare the influence of different cutoffs nc on the result.

#for nc in [2,5,10,20]:
#L, t, U, mu, nc

for mu in mus:
#for g in psi_L100.keys():
    model, psi = dmrg_gs(5, 0.1, 1.0, mu, 4, chi_max=64, eps=1.e-10)
    #psi = psi_L100[g]
    corr = correlation_function(psi, cr, ann, model.L//4)
    corrs[mu] = corr
    #exp_val = psi.site_expectation_value(num)[L//4:]  # only for i >= L//4
    #connected_corr = corr[:] - exp_val[0]* exp_val[:]
    #print("<Sx> =", exp_val[0])
    #conn_corrs[g] = connected_corr
"""
#connected correlations are a different thing 
#dont need conn_corrs right?
#also do need density 
#and plots - especially phase diagram 

"""
# f)
# calculate ground states
#gs = [0.5, 1., 1.05, 1.1, 1.2, 1.5]
#what would we vary? all three t U mu? 
L = 100
psi_L100 = {}
for g in gs:
    print("g=", g)
    model, psi = dmrg_gs(L, 1., g, chi_max=100)
    psi_L100[g] = psi


